In [22]:
import os
import pandas as pd
from rdkit import Chem

DATAPATH = "../data"

In [13]:
##compare that the molecule order is the same based on smiles 
import pandas as pd

# Load DrugBank_DDI_interactions.xls file
df = pd.read_excel('../data/DrugBank_DDI_interactions.xls', sheet_name = '003 - final')
df = df[["Smiles", "Outcome"]]

mols = [Chem.MolFromSmiles(s) for s in df["Smiles"].tolist()]
st_smiles = [Chem.MolToSmiles(mol) if mol is not None else None for mol in mols]
inchikey = [Chem.MolToInchiKey(mol) if mol is not None else None for mol in mols]

df["smiles"]= st_smiles
df["inchikey"] = inchikey
print(df.shape)
df.dropna(subset="smiles", inplace=True)
df.rename(columns={"Outcome":"outcome"}, inplace=True)
df = df[["smiles", "inchikey", "outcome"]]
print(df.shape)

df.to_csv("../data/training_set.csv", index=False)

(591, 4)
(591, 3)


# Data Analysis

In [23]:
#TDC dataset
df1 = pd.read_csv('../data/dili_tdc_dataset.csv')
#Our dataset
df2 = pd.read_csv('../data/training_set.csv')

mols = [Chem.MolFromSmiles(s) for s in df1["Drug"].tolist()]
st_smiles = [Chem.MolToSmiles(mol) if mol is not None else None for mol in mols]
inchikey = [Chem.MolToInchiKey(mol) if mol is not None else None for mol in mols]
df1["smiles"]= st_smiles
df1["inchikey"] = inchikey
print(df1.shape)
df1.dropna(subset="smiles", inplace=True)
df1.rename(columns={"Y":"outcome"}, inplace=True)
df1 = df1[["smiles", "inchikey", "outcome"]]
print(df1.shape)

# Merge df1 and df2 on 'InChIKey' column with 'left' join
merged_df = pd.merge(df1, df2[['inchikey']], on='inchikey', how='left', indicator=True)

# Filter out rows present in df2
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column '_merge'
filtered_df = filtered_df.drop(columns='_merge')
filtered_df.to_csv(os.path.join(DATAPATH, "tdc_test_set.csv"), index=False)

(475, 5)
(475, 3)


In [21]:
overlap_smiles = set(df1['inchikey']).intersection(set(df2['inchikey']))
len(overlap_smiles)

191

In [25]:
# Count the occurrences of positive (1) and negative (0) outcomes in the 'outcome' column
outcome_counts = df2['outcome'].value_counts()

# Print the result
total_samples = len(df2)
print("Number of Negative (0) outcomes:", outcome_counts[0])
print("Number of Positive (1) outcomes:", outcome_counts[1])
print("Total number of samples:", total_samples)

# Calculate and print the ratio of negative (0) and positive (1) outcomes
outcome_ratio = df2['outcome'].value_counts(normalize=True)
print("\nRatio of Negative (0) outcomes:", outcome_ratio[0])
print("Ratio of Positive (1) outcomes:", outcome_ratio[1])


Number of Negative (0) outcomes: 194
Number of Positive (1) outcomes: 397
Total number of samples: 591

Ratio of Negative (0) outcomes: 0.32825719120135366
Ratio of Positive (1) outcomes: 0.6717428087986463
